# **Magenta**
Ce notebook se base sur Magenta, un projet de recherche open source des équipes de Google Brain explorant le rôle du machine learning dans le processus de création artistique et musicale. 


#Installation

Installation de Magenta et de ses dépendances.

In [13]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

Installing dependencies...


In [14]:
# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta #<========== ! 
import note_seq
import tensorflow

print('🎉 Done!')
print(magenta.__version__)
print(tensorflow.__version__)

Importing libraries and defining some helper functions...
🎉 Done!
2.1.3
2.6.0


# Créer des sons ...

Magenta utilise la classe NoteSequence() comme représentation des fichiers MIDI dans son environnement. Introduction aux fichiers MIDI

In [15]:
from note_seq.protobuf import music_pb2

twinkle_twinkle = music_pb2.NoteSequence()
# Add the notes to the sequence.
twinkle_twinkle.notes.add(pitch=60, start_time=0.0, end_time=0.5, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=0.5, end_time=1.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.0, end_time=1.5, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=1.5, end_time=2.0, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.0, end_time=2.5, velocity=80)
twinkle_twinkle.notes.add(pitch=69, start_time=2.5, end_time=3.0, velocity=80)
twinkle_twinkle.notes.add(pitch=67, start_time=3.0, end_time=4.0, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.0, end_time=4.5, velocity=80)
twinkle_twinkle.notes.add(pitch=65, start_time=4.5, end_time=5.0, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.0, end_time=5.5, velocity=80)
twinkle_twinkle.notes.add(pitch=64, start_time=5.5, end_time=6.0, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.0, end_time=6.5, velocity=80)
twinkle_twinkle.notes.add(pitch=62, start_time=6.5, end_time=7.0, velocity=80)
twinkle_twinkle.notes.add(pitch=60, start_time=7.0, end_time=8.0, velocity=80) 
twinkle_twinkle.total_time = 8

twinkle_twinkle.tempos.add(qpm=60);

# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(twinkle_twinkle)
# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

# Here's another NoteSequence!
teapot = music_pb2.NoteSequence()
teapot.notes.add(pitch=69, start_time=0, end_time=0.5, velocity=40)
teapot.notes.add(pitch=71, start_time=0.5, end_time=1, velocity=50)
teapot.notes.add(pitch=73, start_time=1, end_time=1.5, velocity=60)
teapot.notes.add(pitch=74, start_time=1.5, end_time=2, velocity=70)
teapot.notes.add(pitch=76, start_time=2, end_time=2.5, velocity=80)
teapot.notes.add(pitch=81, start_time=3, end_time=4, velocity=80)
teapot.notes.add(pitch=78, start_time=4, end_time=5, velocity=80)
teapot.notes.add(pitch=81, start_time=5, end_time=6, velocity=80)
teapot.notes.add(pitch=76, start_time=6, end_time=8, velocity=80)
teapot.total_time = 8

teapot.tempos.add(qpm=60);

note_seq.plot_sequence(teapot)
note_seq.play_sequence(teapot,synth=note_seq.fluidsynth)

#Enregistrement en fichiers MIDI


Exercice : Créez votre propre fichier midi ! 

In [16]:
#Code here

Il est possible d'importer et de convertir un fichier MIDI en NoteSequence() et inversement

In [17]:
#NoteSequence() to MIDI
note_seq.sequence_proto_to_midi_file(twinkle_twinkle, 'twinkle_twinkle.mid')
note_seq.sequence_proto_to_midi_file(teapot,'teapot.mid')
#Téléchargement
files.download('twinkle_twinkle.mid') 
files.download('teapot.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
#MIDI to NoteSequence()
twtw = note_seq.midi_io.midi_file_to_note_sequence('twinkle_twinkle.mid')
note_seq.plot_sequence(twtw)
note_seq.play_sequence(twtw,synth=note_seq.fluidsynth)

# ... à partir des réseaux de neurones
Les réseaux de neurones sont des outils puissants permettant d'apprendre et modéliser une distribution de donnée. En d'autres termes, les liens Ainsi, un modèle adéquat permettrait de générer des musiques semblables à celles vues par le modèle durant son entraînement.

## MelodyRNN

# Step 2. Using Machine Learning to make music

`note_seq` has several Machine Learning models, each with different strengths. All models are built with [Tensorflow](https://www.tensorflow.org), so they will run faster if you can run them on a GPU. Here are some of the most popular ones:

- [**MelodyRNN**](https://github.com/magenta/magenta/tree/master/magenta/models/melody_rnn) - you give it a NoteSequence, and it continues it in the style of your original NoteSequence.
- [**MusicVAE**](https://github.com/magenta/magenta/tree/master/magenta/models/music_vae) - generates brand new NoteSequences or interpolates between two sequences.
- [**Onsets and Frames**](https://github.com/magenta/magenta/tree/master/magenta/models/onsets_frames_transcription) -- transcribes piano audio

Now that we know how to use `NoteSequences`, adding some basic Machine Learning is a continuation of that. The pattern for using any of these models is:

- Load `note_seq` (which we already know how to do!)
- Create a model from a downloaded checkpoint (i.e. where the weights, or the encoding, of the model lives)
- Ask the model to do something.

## Melody RNN

A MelodyRNN is an [LSTM-based](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) language model for musical notes -- it is best at continuing a NoteSequence that you give it.

To use it, you need to give it a sequence to continue and the model will return the following sequence.

This example shows how to use the basic Melody RNN model -- check out the [docs](https://github.com/magenta/magenta/tree/master/magenta/models/melody_rnn) for the other models, such as `lookback_rnn` and `attention_rnn`.

### Initialize the model

In [19]:
# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

In [20]:
#4 choix de configurations : basic_rnn, mono_rnn, lookback_rnn, attention_rnn
config = 'basic_rnn'

print('Downloading model bundle. This will take less than a minute...')
note_seq.notebook_utils.download_bundle(config+'.mag', '/content/')

# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/{}.mag'.format(config))
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map[config](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print('🎉 Done!')

Initializing Melody RNN...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmp1t7oja2y/model.ckpt
🎉 Done!


### Continuer une séquence de note

Avec **Melody RNN**, vous pouvez configurer le nombre de pas de la nouvelle séquence, ainsi que la "température" du résultat -- plus la température est élevée, plus votre séquence sera aléatoire (et moins elle ressemblera à celle d'entrée). Vous pouvez jouer avec ces valeurs et visualiser la différence entre les séquences résultantes.

In [21]:
# Model options. Change these to get different generated sequences! 
input_sequence = twinkle_twinkle # change this to teapot if you want
num_steps = 128 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence

In [22]:
# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -104.838249 


## Music VAE

Le modèle [MusicVAE](https://g.co/magenta/musicvae) permet de créer de nouvelles séquences de notes et même d'interpoler une nouvelle séquence à partir de deux d'entre elles. Le modèle permet de compacter l'information en entrée et de recréer cette information à partir de cet espace compact.


Initialisation du modèle

In [23]:
print('Copying checkpoint from GCS. This will take less than a minute...')
# This will download the mel_2bar_big checkpoint. There are more checkpoints that you
# can use with this model, depending on what kind of output you want
# See the list of checkpoints: https://github.com/magenta/magenta/tree/master/magenta/models/music_vae#pre-trained-checkpoints
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# Import dependencies.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Initialize the model.
print("Initializing Music VAE...")
music_vae = TrainedModel(
      configs.CONFIG_MAP['cat-mel_2bar_big'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')

print('🎉 Done!')

Copying checkpoint from GCS. This will take less than a minute...
Initializing Music VAE...
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Restoring parameters from /content/mel_2bar_big.ckpt
🎉 Done!


### Créer de nouvelles séquences

In [24]:
#Config
n = 2 #Nombre de séquence à générer
length = 80 #longueur de la séquence à générer
temperature= 1.0 #degré de random de la génération

In [25]:
generated_sequences = music_vae.sample(n=n, length=length, temperature=temperature)

for ns in generated_sequences:
  # print(ns)
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

### Interpolating between two sequences

In [32]:
num_steps = 8
length = 32
sequence2interpol = [twinkle_twinkle, teapot]

In [33]:
# We're going to interpolate between the Twinkle Twinkle Little Star
# NoteSequence we defined in the first section, and one of the generated
# sequences from the previous VAE example

# How many sequences, including the start and end ones, to generate.
num_steps = 8;

# This gives us a list of sequences.
note_sequences = music_vae.interpolate(
      sequence2interpol[0],
      sequence2interpol[1], 
      num_steps=num_steps,
      length=32)

# Concatenate them into one long sequence, with the start and 
# end sequences at each end. 
interp_seq = note_seq.sequences_lib.concatenate_sequences(note_sequences)

note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)
note_seq.plot_sequence(interp_seq)

# That's it!

You're now ready to build your own amazing, Machine Learning powered, music instrument! If you want more information, you can check out:

- some [demos](https://magenta.tensorflow.org/demos) `#MadeWithMagenta`
- some more of our [Colab notebooks](https://magenta.tensorflow.org/demos#colab-notebooks)
- the [documentation](https://github.com/magenta/magenta)
- the [Magenta blog](https://magenta.tensorflow.org/blog), which talks about all the mathy bits we skipped.

Have fun! 💕